In [2]:
import opendatasets as od
od.download("https://www.kaggle.com/jerimee/north-carolina-voter-file")

Skipping, found downloaded files in ".\north-carolina-voter-file" (use force=True to force download)


In [3]:
import pandas as pd

nc = "north-carolina-voter-file\\ncvoter_Statewide\\ncvoter_Statewide.txt"
nc = pd.read_csv(nc, delimiter = '\t')

C:\Users\KNF\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (24,33,34,35,36,37,38,40,41,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,69,70) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
for col in nc.columns:
    print(col)

county_id
county_desc
voter_reg_num
status_cd
voter_status_desc
reason_cd
voter_status_reason_desc
absent_ind
name_prefx_cd
last_name
first_name
middle_name
name_suffix_lbl
res_street_address
res_city_desc
state_cd
zip_code
mail_addr1
mail_addr2
mail_addr3
mail_addr4
mail_city
mail_state
mail_zipcode
full_phone_number
race_code
ethnic_code
party_cd
gender_code
birth_age
birth_state
drivers_lic
registr_dt
precinct_abbrv
precinct_desc
municipality_abbrv
municipality_desc
ward_abbrv
ward_desc
cong_dist_abbrv
super_court_abbrv
judic_dist_abbrv
nc_senate_abbrv
nc_house_abbrv
county_commiss_abbrv
county_commiss_desc
township_abbrv
township_desc
school_dist_abbrv
school_dist_desc
fire_dist_abbrv
fire_dist_desc
water_dist_abbrv
water_dist_desc
sewer_dist_abbrv
sewer_dist_desc
sanit_dist_abbrv
sanit_dist_desc
rescue_dist_abbrv
rescue_dist_desc
munic_dist_abbrv
munic_dist_desc
dist_1_abbrv
dist_1_desc
dist_2_abbrv
dist_2_desc
confidential_ind
birth_year
ncid
vtd_abbrv
vtd_desc


In [72]:
cols = ['cong_dist_abbrv',
'party_cd',
'status_cd',
'precinct_abbrv']

nc_voters = nc[cols]
nc_voters.head()

,cong_dist_abbrv,party_cd,status_cd,precinct_abbrv
0,13.0,UNA,A,08N
1,13.0,UNA,A,10N
2,13.0,UNA,A,03S
3,13.0,UNA,A,124
4,13.0,DEM,A,08N


In [73]:
nc_voters['party_cd'].unique()

#NC Parties = Undeclared, Constitution, Democratic, Green, Libertarian and Republican
#To match our other data, I'm changing to OTH, CONST, DEM, GRN, LBT, and REP

nc_voters['party_cd'] = nc_voters['party_cd'].replace(['UNA','DEM','REP','LIB','GRE','CST'],['OTH','DEM','REP','LBT','GRN','CONST'])

<ipython-input-73-f6446cbe4359>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nc_voters['party_cd'] = nc_voters['party_cd'].replace(['UNA','DEM','REP','LIB','GRE','CST'],['OTH','DEM','REP','LBT','GRN','CONST'])


In [74]:
#keeping only active voters
nc_voters = nc_voters[nc_voters['status_cd']=='A']
nc_voters = nc_voters.drop(columns='status_cd', axis=1)

In [145]:
#Aggregate by county/precinct/party
nc_totals = nc_voters.groupby(['cong_dist_abbrv', 'precinct_abbrv','party_cd']).size().reset_index(name='counts')

In [146]:
nc_totals['STATE']='NC'
nc_totals.head()

,cong_dist_abbrv,precinct_abbrv,party_cd,counts,STATE
0,1.0,1.0,DEM,838,NC
1,1.0,1.0,LBT,8,NC
2,1.0,1.0,OTH,563,NC
3,1.0,1.0,REP,880,NC
4,1.0,2.0,CONST,2,NC


In [147]:
nc_totals = nc_totals.rename(columns={'cong_dist_abbrv':'DISTRICT', 'precinct_abbrv':'PRECINCT','party_cd':'PARTY','counts':'VOTERS'})
nc_totals = nc_totals[['STATE','PRECINCT','DISTRICT','PARTY','VOTERS']]

In [171]:
# Inspo from: https://www.youtube.com/watch?v=iOurqfCJwy8
# but I changed it so I didn't have to explicitly call the columns in the print statement

l = []
for col in nc_totals.columns:
    x=(col)
    l.append(x)

with open('ncinsert.sql','w'):
    for index, row in nc_totals.iterrows():
        print('INSERT INTO precinct'+'('+str(l).strip('[]')+') '+'VALUES('+
              '"'+str(row[str(l[0])])+'", '+
              '"'+str(row[str(l[1])])+'", '+
              '"'+str(row[str(l[2])])+'", '+
              '"'+str(row[str(l[3])])+'", '+
              str(row[str(l[4])])+
              ');',file=open('ncinsert.sql','a'))

#Adding the begin transactoin and commit lines to the original file
with open('ncinsert.sql', 'r') as original: data = original.read()
with open('ncinsert.sql', 'w') as modified: modified.write("BEGIN TRANSACTION;\n" + data + "COMMIT;")

In [174]:
od.download("https://www.kaggle.com/gabrielaltay/georgia-voter-list-202011")

C:\Users\KNF\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (4,9,10,11,39) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [ ]:
path = "georgia-voter-list-202011\\tbl_prod_GABU202012_all.csv"
ga = pd.read_csv(path, sep='|', index_col=False)

In [176]:
for col in ga.columns:
    print(col)

county_code
registration_number
voter_status
residence_city
residence_zipcode
birthyear
registration_date
race
gender
land_district
land_lot
status_reason
county_precinct_id
city_precinct_id
congressional_district
senate_district
house_district
judicial_district
commission_district
school_district
county_districta_name
county_districta_value
county_districtb_name
county_districtb_value
municipal_name
municipal_code
ward_city_council_name
ward_city_council_code
city_school_district_name
city_school_district_value
city_dista_name
city_dista_value
city_distb_name
city_distb_value
city_distc_name
city_distc_value
city_distd_name
city_distd_value
date_last_voted
party_last_voted
date_added
date_changed
district_combo
race_desc
last_contact_date


In [ ]:
cols=['voter_status',
     'congressional_district',
     ]
ga = 